# An investigation of PyLinac's MV kV iso code

From https://github.com/jrkerns/pylinac/blob/95d3ea6b8f853beb4c9729f36b5451bbc4e7e2a7/pylinac/winston_lutz.py

PyLinac has the following license:

Modifications have been made to the PyLinac code. Please see the original repository (https://github.com/jrkerns/pylinac) for the original code.

In [ ]:
import os
import datetime
from glob import glob
from collections import namedtuple

import numpy as np
import pandas as pd
import scipy.ndimage

import pydicom
from pylinac import WinstonLutz

In [ ]:
data_root = r'S:\Physics\Programming\data\MVISO'

In [ ]:
data_record = glob(os.path.join(data_root, 'iView*.xlsx'))[0]
dicom_files = np.array(glob(os.path.join(data_root, '*.dcm')))

In [ ]:
record = pd.read_excel(data_record, skiprows=4)
timestamps_initial = record['Datetime']
timestamps = timestamps_initial[timestamps_initial.notnull()].values
gantry = record['Gantry'][timestamps_initial.notnull()].values
colimator = record['Col'][timestamps_initial.notnull()].values
turntable = record['TT'][timestamps_initial.notnull()].values
beam = record['Energy'][timestamps_initial.notnull()].values

In [ ]:
datasets = np.array([
    pydicom.read_file(dicom_file, force=True)
    for dicom_file in dicom_files
])

In [ ]:
# np.random.shuffle(datasets)

In [ ]:
acquisition_datetimes = np.array([
    datetime.datetime.strptime(dataset.AcquisitionDate + dataset.AcquisitionTime, '%Y%m%d%H%M%S.%f')
    for dataset in datasets
], dtype=np.datetime64)

In [ ]:
diff_map = np.abs(acquisition_datetimes[None,:] - timestamps[:, None]) < np.timedelta64(2, 's')
timestamp_index, acquisition_index = np.where(diff_map)

In [ ]:
assert len(set(acquisition_index)) == len(acquisition_index)
assert len(acquisition_index) == len(acquisition_datetimes)

In [ ]:
datasets = datasets[acquisition_index]
dicom_files = dicom_files[acquisition_index]
timestamps = timestamps[timestamp_index]
gantry = gantry[timestamp_index]
colimator = colimator[timestamp_index]
turntable = turntable[timestamp_index]
beam = beam[timestamp_index]

acquisition_datetimes = np.array([
    datetime.datetime.strptime(dataset.AcquisitionDate + dataset.AcquisitionTime, '%Y%m%d%H%M%S.%f')
    for dataset in datasets
], dtype=np.datetime64)

diff_map = np.abs(acquisition_datetimes[None,:] - timestamps[:, None]) < np.timedelta64(2, 's')
timestamp_index, acquisition_index = np.where(diff_map)

assert np.all(timestamp_index == acquisition_index)

In [ ]:
pixel_arrays = [
    dataset.pixel_array
    for dataset in datasets
]

In [ ]:
# https://github.com/jrkerns/pylinac/blob/95d3ea6b8f853beb4c9729f36b5451bbc4e7e2a7/pylinac/core/image.py#L358-L377
    
def crop(pixel_array, pixels):    
    pixel_array = pixel_array[pixels:, :]
    pixel_array = pixel_array[:-pixels, :]
    pixel_array = pixel_array[:, pixels:]
    pixel_array = pixel_array[:, :-pixels]

In [ ]:
# https://github.com/jrkerns/pylinac/blob/95d3ea6b8f853beb4c9729f36b5451bbc4e7e2a7/pylinac/winston_lutz.py#L570-L591

def clean_edges(pixel_array, window_size):
    
    def has_noise(pixel_array, window_size):
        near_min, near_max = np.percentile(pixel_array, [5, 99.5])
        img_range = near_max - near_min
        
        top = pixel_array[:window_size, :]
        left = pixel_array[:, :window_size]
        bottom = pixel_array[-window_size:, :]
        right = pixel_array[:, -window_size:]
        
        edge_array = np.concatenate((top.flatten(), left.flatten(), bottom.flatten(), right.flatten()))
        edge_too_low = edge_array.min() < (near_min - img_range / 10)
        edge_too_high = edge_array.max() > (near_max + img_range / 10)
        
        return edge_too_low or edge_too_high

    safety_stop = np.min(pixel_array.shape)/10
    
    while has_noise(pixel_array, window_size) and safety_stop > 0:
        crop(pixel_array, window_size)
        safety_stop -= 1

In [ ]:
# https://github.com/jrkerns/pylinac/blob/95d3ea6b8f853beb4c9729f36b5451bbc4e7e2a7/pylinac/core/image.py#L446-L459

def as_binary(pixel_array, threshold):
    return np.where(pixel_array >= threshold, 1, 0)

In [ ]:
Point = namedtuple('x', 'y')

In [ ]:
# https://github.com/jrkerns/pylinac/blob/95d3ea6b8f853beb4c9729f36b5451bbc4e7e2a7/pylinac/winston_lutz.py#L593-L614

def find_field_centroid(pixel_array):
    min, max = np.percentile(pixel_array, [5, 99.9])
    threshold_array = as_binary(pixel_array, (max - min)/2 + min)

    cleaned_img = scipy.ndimage.binary_erosion(threshold_array)
    [*edges] = bounding_box(cleaned_img)
    edges[0] -= 10
    edges[1] += 10
    edges[2] -= 10
    edges[3] += 10
    coords = scipy.ndimage.measurements.center_of_mass(threshold_img)
    p = Point(x=coords[-1], y=coords[0])

    return p, edges

In [ ]:
# https://github.com/jrkerns/pylinac/blob/95d3ea6b8f853beb4c9729f36b5451bbc4e7e2a7/pylinac/core/profile.py#L250-L307
    
def penumbra_point(self, side: str='left', x: int=50, interpolate: bool=False, kind: str='index'):
    # get peak
    peak = copy.copy(self._initial_peak_idx)
    peak = int(peak*self.interpolation_factor if interpolate else peak)

    # get y-data
    if side == LEFT:
        y_data = self._values_left_interp if interpolate else self._values_left
    else:
        y_data = self._values_right_interp if interpolate else self._values_right

    # get threshold
    max_point = y_data.max()
    threshold = max_point * (x / 100)

    # find the index, moving 1 element at a time until the value is encountered
    found = False
    at_end = False
    try:
        while not found and not at_end:
            if y_data[peak] < threshold:
                found = True
                peak -= 1 if side == RIGHT else -1
            elif peak == 0:
                at_end = True
            peak += 1 if side == RIGHT else -1
    except IndexError:
        raise IndexError("The point of interest was beyond the profile; i.e. the profile may be cut off on the side")

    if kind == VALUE:
        return self._values_interp[peak] if interpolate else self.values[peak]
    elif kind == INDEX:
        if interpolate:
            peak /= self.interpolation_factor
        return peak

In [ ]:
# https://github.com/jrkerns/pylinac/blob/95d3ea6b8f853beb4c9729f36b5451bbc4e7e2a7/pylinac/core/profile.py#L343-L362

def fwxm(self, x: int=50, interpolate: bool=False) -> float:
    li = self._penumbra_point(LEFT, x, interpolate)
    ri = self._penumbra_point(RIGHT, x, interpolate)
    fwxm = np.abs(ri - li)
    return fwxm

In [ ]:
# https://github.com/jrkerns/pylinac/blob/95d3ea6b8f853beb4c9729f36b5451bbc4e7e2a7/pylinac/core/profile.py#L364-L379
    
def fwxm_center(pixel_array, x: int=50, interpolate: bool=False, kind: str='index') -> float:
    """Return the center index of the FWXM.
    See Also
    --------
    fwxm() : Further parameter info
    """
    fwxm = self.fwxm(x, interpolate=interpolate)
    li = self._penumbra_point(LEFT, x, interpolate)
    fwxmcen = np.abs(li + fwxm / 2)
    if not interpolate:
        fwxmcen = int(round(fwxmcen))
    if kind == VALUE:
        return self.values[fwxmcen] if not interpolate else self._values_interp[int(fwxmcen*self.interpolation_factor)]
    else:
        return fwxmcen

In [ ]:
# https://github.com/jrkerns/pylinac/blob/95d3ea6b8f853beb4c9729f36b5451bbc4e7e2a7/pylinac/core/image.py#L397-L400

def invert(pixel_array):
    return -pixel_array + pixel_array.max() + pixel_array.min()

In [ ]:
# https://github.com/jrkerns/pylinac/blob/95d3ea6b8f853beb4c9729f36b5451bbc4e7e2a7/pylinac/winston_lutz.py#L616-L659

def find_bb(pixel_array):
    # get initial starting conditions
    hmin, hmax = np.percentile(pixel_array, [5, 99.9])
    spread = hmax - hmin
    max_thresh = hmax
    lower_thresh = hmax - spread / 1.5
    # search for the BB by iteratively lowering the low-pass threshold value until the BB is found.
    found = False
    while not found:
        try:
            binary_arr = np.logical_and((max_thresh > pixel_array), (pixel_array >= lower_thresh))
            labeled_arr, num_roi = ndimage.measurements.label(binary_arr)
            roi_sizes, bin_edges = np.histogram(labeled_arr, bins=num_roi + 1)
            bw_bb_img = np.where(labeled_arr == np.argsort(roi_sizes)[-3], 1, 0)

            if not is_round(bw_bb_img):
                raise ValueError
            if not is_modest_size(bw_bb_img, find_field_centroid(pixel_array)):
                raise ValueError
            if not is_symmetric(bw_bb_img):
                raise ValueError
        except (IndexError, ValueError):
            max_thresh -= 0.05 * spread
            if max_thresh < hmin:
                raise ValueError("Unable to locate the BB. Make sure the field edges do not obscure the BB and that there is no artifacts in the images.")
        else:
            found = True

    # determine the center of mass of the BB
    inv_img = invert(pixel_array)
    
    x_arr = np.abs(np.average(bw_bb_img, weights=inv_img, axis=0))
    x_com = SingleProfile(x_arr).fwxm_center(interpolate=True)
    y_arr = np.abs(np.average(bw_bb_img, weights=inv_img, axis=1))
    y_com = SingleProfile(y_arr).fwxm_center(interpolate=True)
    
    return Point(x_com, y_com)

In [ ]:
diff_map

In [ ]:
diff_map

In [ ]:
acquisition_datetimes[29]
timestamps.values[0]

In [ ]:
np.timedelta64(1, 's')

In [ ]:
acquisition_times

In [ ]:
np.array(timestamps.values)